In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audios/audios_waves/sea waves/2-77945-B.wav
/kaggle/input/audios/audios_waves/sea waves/1-91359-A.wav
/kaggle/input/audios/audios_waves/sea waves/5-219379-C.wav
/kaggle/input/audios/audios_waves/sea waves/3-118658-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-182613-B.wav
/kaggle/input/audios/audios_waves/sea waves/3-164120-A.wav
/kaggle/input/audios/audios_waves/sea waves/2-50668-A.wav
/kaggle/input/audios/audios_waves/sea waves/1-28135-A.wav
/kaggle/input/audios/audios_waves/sea waves/5-216370-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-169127-B.wav
/kaggle/input/audios/audios_waves/sea waves/1-47250-B.wav
/kaggle/input/audios/audios_waves/sea waves/4-157611-B.wav
/kaggle/input/audios/audios_waves/sea waves/1-61252-A.wav
/kaggle/input/audios/audios_waves/sea waves/3-155642-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-165823-A.wav
/kaggle/input/audios/audios_waves/sea waves/4-157611-A.wav
/kaggle/input/audios/audios_waves/sea waves/1-39901-A.wav
/kag

In [31]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
import librosa
import librosa
import soundfile as sf

In [32]:
# Now, X contains the extracted MFCC features, and y contains corresponding labels.
def process_audio_folder(folder_path):
    features = []
    labels = []

    for label, animal_folder in enumerate(os.listdir(folder_path)):
        for audio_file in os.listdir(os.path.join(folder_path, animal_folder)):
            audio_path = os.path.join(folder_path, animal_folder, audio_file)

            # Extract MFCC features
            mfccs, chroma, mel, contrast, tonnetz = extract_features(audio_path)
#             mfccs, mel = extract_features(audio_path)
#             mfccs, contrast = extract_features(audio_path)
            #mfccs, tonnetz = extract_features(audio_path)
#             mfccs,mel, tonnetz = extract_features(audio_path)
            if mfccs is not None:
                all_features = np.concatenate([mfccs, chroma, mel, contrast, tonnetz])
#                 all_features = np.concatenate([mfccs, chroma])
#                 all_features = np.concatenate([mfccs, mel])
                #all_features = np.concatenate([mfccs, contrast])
#                 all_features = np.concatenate([mfccs, tonnetz])
#                 all_features = np.concatenate([mel, tonnetz])
                features.append(all_features)
                labels.append(label)

    return np.array(features), np.array(labels)

In [33]:
def extract_features(file_name=None, normalize=True, cutoff_frequency=None):
    if file_name:
        print('Extracting', file_name)
        X, sample_rate = sf.read(file_name, dtype='float32')
    if X.ndim > 1:
        X = X[:, 0]
    X = X.T

    # Apply normalization
    if normalize:
        X = librosa.util.normalize(X)

    # Apply high-pass filter if cutoff frequency is specified
    if cutoff_frequency:
        X = high_pass_filter(X, sample_rate, cutoff_frequency)

    # Short-term Fourier transform
    stft = np.abs(librosa.stft(X))

    # MFCC (Mel-frequency cepstrum)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=192).T, axis=0)

    # Chroma
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)

    # Mel spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)

    # Spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)

    # Tonnetz
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)

    return mfccs, chroma, mel, contrast, tonnetz

#     return  chroma, mfccs
#     return  mfccs, mel
#     return mfccs,contrast
#     return mfccs,tonnetz
#     return mfccs,mel,tonnetz

In [34]:
# Additional function for high-pass filter
def high_pass_filter(signal, sample_rate, cutoff_frequency):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(1, normal_cutoff, btype='high', analog=False)
    filtered_signal = lfilter(b, a, signal)
    return filtered_signal

In [35]:

# Example usage:
folder_path = "/kaggle/input/audios/audios_waves/"
X, y = process_audio_folder(folder_path)

Extracting /kaggle/input/audios/audios_waves/sea waves/2-77945-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-91359-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/5-219379-C.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/3-118658-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/4-182613-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/3-164120-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/2-50668-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-28135-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/5-216370-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/4-169127-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-47250-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/4-157611-B.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/1-61252-A.wav
Extracting /kaggle/input/audios/audios_waves/sea waves/3-155642-A.wav
Extracting /kaggle/input/a

In [91]:
print(X)
print(y)

[[-2.27249002e+00  6.73703918e+01 -4.00791588e+01 ...  2.40626032e-02
   2.18425073e-03 -2.10040039e-02]
 [-5.63411026e+01  1.44255508e+02 -4.07874413e+01 ...  7.09809247e-03
   1.25271965e-02 -3.98982854e-03]
 [-1.19049210e+02  1.80238678e+02 -9.85047455e+01 ... -6.14604352e-03
  -1.91698286e-03 -4.21795500e-04]
 ...
 [-5.60843506e+02  3.72460861e+01 -1.29059544e+01 ...  1.90752047e-02
   1.70570564e-02 -3.08373323e-03]
 [-4.73198975e+02  3.75605278e+01 -9.35215950e+00 ... -1.70691318e-02
   1.16746539e-02  7.18639060e-03]
 [-4.56609833e+02  5.18358612e+01 -8.53918266e+00 ... -3.07373032e-03
   7.09097780e-03 -5.68949947e-03]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3

In [93]:
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# # Assuming you have your features X and labels y
# # X, y = ...

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the Multinomial Naive Bayes model
# nb_model = MultinomialNB()

# # Train the model
# nb_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = nb_model.predict(X_test)

# # Evaluate the performance
# accuracy = accuracy_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred)
# classification_rep = classification_report(y_test, y_pred)

# # Display the results
# print(f'Accuracy: {accuracy}')
# print(f'Confusion Matrix:\n{conf_matrix}')
# print(f'Classification Report:\n{classification_rep}')



In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


[](http://)

In [95]:

# Step 1: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
y_pred = y_test


In [96]:
# Step 2: Data Normalization
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)



# aucurancy = 69

In [97]:
# Step 3: Building the Neural Network
model = Sequential()
aucurancy = 69
model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))



In [68]:
# model = Sequential()
# model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='sigmoid'))
# model.add(Dense(64, activation='tanh'))

Test Accuracy: 0.06395348906517029

# #aucurancy = 65


In [ ]:
#aucurancy = 65
# model.add(Dense(256, input_dim=X_train_normalized.shape[1], activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(len(np.unique(y)), activation='softmax'))

# aucurancy = 68

In [ ]:
#aucurancy = 68
# model.add(Dense(256, input_dim=X_train_normalized.shape[1], activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(len(np.unique(y)), activation='softmax'))

# adding Dense layer => Test Accuracy: 0.08139535039663315

In [53]:
# model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='sigmoid'))
# model.add(Dense(64, activation='tanh'))

In [59]:
# Add the first layer with a specific activation function
# model = Sequential()
# model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='relu'))

# # Add the second layer with a different activation function
# model.add(Dense(64, activation='tanh'))

# # Add the output layer with another activation function
# model.add(Dense(len(np.unique(y)), activation='softmax'))

Test Accuracy: 0.6686046719551086

In [64]:
# model = Sequential()
# model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='softmax'))

# # Add the second layer with a different activation function
# model.add(Dense(64, activation='softmax'))

# # Add the output layer with another activation function
# model.add(Dense(len(np.unique(y)), activation='softmax'))

Test Accuracy: 0.22674418985843658

In [83]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

# # Create a Sequential model
# model = Sequential()

# # Add the first layer with dropout
# model.add(Dense(128, input_dim=X_train_normalized.shape[1], activation='relu'))
# model.add(Dropout(0.5))  # Adding dropout with a dropout rate of 0.5

# # Add the second layer with dropout
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))  # Adding dropout with a dropout rate of 0.5

# # Add the output layer
# model.add(Dense(len(np.unique(y)), activation='softmax'))

# Test Accuracy: 0.5581395626068115

In [99]:
# Step 4: Model Compilation
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [100]:

# Step 5: Model Training
model.fit(X_train_normalized, y_train, epochs=40, batch_size=32, validation_split=0.2)



Epoch 1/40
7/7 [==============================] - 1s 36ms/step - loss: 2.1931 - accuracy: 0.1471 - val_loss: 2.1167 - val_accuracy: 0.1923
Epoch 2/40
7/7 [==============================] - 0s 10ms/step - loss: 2.0457 - accuracy: 0.2500 - val_loss: 2.0534 - val_accuracy: 0.2115
Epoch 3/40
7/7 [==============================] - 0s 8ms/step - loss: 1.9557 - accuracy: 0.3627 - val_loss: 1.9900 - val_accuracy: 0.2885
Epoch 4/40
7/7 [==============================] - 0s 8ms/step - loss: 1.8470 - accuracy: 0.3775 - val_loss: 1.9404 - val_accuracy: 0.2692
Epoch 5/40
7/7 [==============================] - 0s 7ms/step - loss: 1.7549 - accuracy: 0.4412 - val_loss: 1.8466 - val_accuracy: 0.3077
Epoch 6/40
7/7 [==============================] - 0s 8ms/step - loss: 1.6389 - accuracy: 0.4265 - val_loss: 1.7772 - val_accuracy: 0.2885
Epoch 7/40
7/7 [==============================] - 0s 10ms/step - loss: 1.5519 - accuracy: 0.4559 - val_loss: 1.6823 - val_accuracy: 0.3654
Epoch 8/40
7/7 [===============

In [101]:
# Step 6: Model Evaluation
accuracy = model.evaluate(X_test_normalized, y_test)[1]
print(f'Test Accuracy: {accuracy}')


6/6 [==============================] - 0s 3ms/step - loss: 0.9139 - accuracy: 0.6977
Test Accuracy: 0.6976743936538696


# best Test Accuracy  0.6976743936538696  all features  mfccs, chroma, mel, contrast, tonnetz  test_size=0.4  epochs=40 son pretraitement 
 

Test Accuracy: 0.6976743936538696   features  mfccs, chroma  test_size=0.4  epochs=40

Test Accuracy: 0.6627907156944275    features  mfccs, mel  test_size=0.4  epochs=40

Test Accuracy: 0.6627907156944275     features  mfccs, contrast  test_size=0.4  epochs=40


Test Accuracy: 0.6511628031730652 features  mfccs, contrast  test_size=0.4  epochs=192



Test Accuracy: 0.6511628031730652 features  mfccs, contrast  test_size=0.4  epochs=40 batch_size=80



Test Accuracy:  0.6395348906517029 features  mfccs, tonnetz  test_size=0.4  epochs=40 batch_size=32


Test Accuracy: 0.3604651093482971 features  chroma, tonnetz  test_size=0.4  epochs=40 batch_size=32

Test Accuracy: 0.3895348906517029 features  mel, tonnetz  test_size=0.4  epochs=40 batch_size=32


In [87]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have your features X and labels y
# X, y = ...

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gaussian Naive Bayes model
nb_model = GaussianNB()

# Train the model
nb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_model.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Display the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 0.627906976744186
Confusion Matrix:
[[ 6  0 10  3  0  0  5  0  0]
 [ 0  5  0  0  1  0  0  1  0]
 [ 4  0  3  1  0  0  0  0  0]
 [ 1  0  0  8  0  1  0  0  1]
 [ 0  0  0  0  4  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0]
 [ 0  0  0  0  0  1  4  0  0]
 [ 0  0  1  0  0  0  0  4  0]
 [ 0  1  0  0  0  1  0  0 15]]
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.25      0.34        24
           1       0.83      0.71      0.77         7
           2       0.21      0.38      0.27         8
           3       0.67      0.73      0.70        11
           4       0.80      1.00      0.89         4
           5       0.62      1.00      0.77         5
           6       0.44      0.80      0.57         5
           7       0.80      0.80      0.80         5
           8       0.94      0.88      0.91        17

    accuracy                           0.63        86
   macro avg       0.65      0.73      0.67        86
weighted a

In [110]:
y_pred_probs = model.predict(X_test)
print(y_pred_probs)
# Convert predicted probabilities to class labels using argmax
y_pred = np.argmax(y_pred_probs, axis=1)

# Convert y_test to the same format as y_pred
# (e.g., if y_test is one-hot encoded, you might need to use argmax)
# Assuming y_test is one-hot encoded, use np.argmax to convert it
y_true = np.argmax(y_test)

6/6 [==============================] - 0s 2ms/step
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.4493449e-30 0.0000000e+00 9.9999994e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  3.6049778e-20 0.0000000e+00]
 [0.0000000e+00 1.2329804e-23 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.3185504e-34]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 8.6609522e-38]]
